In [1]:
import pandas as pd
import numpy as np

from functools import partial
import re
import os
import time
import tensorboard
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import keras
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling1D, Lambda, UpSampling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input,Flatten,Dense, Dropout, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras import models
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
import keras.models
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Activation

In [3]:
def load_files_list(directory):
    '''
    Load the audiofiles inside a directory.
    '''
    return os.listdir(directory)

## Loading the descriptors (1D)

In [4]:
DIRECTORY = '/home/belen_alastruey/PYTHON/Autoencoder/'

In [5]:
path = DIRECTORY + 'MEL/'

In [6]:
files = load_files_list(path)

### Read .npy

In [7]:
descriptors= []
#labels = []
for file in files:
    if "slider" in file: #slider_id00
        descr = np.load(path + file)
        descriptors.append(descr)
        #labels.append(re.split(r'_', file)[1])

In [8]:
descr = np.asarray(descriptors)

In [9]:
descr.shape

(2670, 128, 216)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test= train_test_split(descr,test_size=0.15, random_state=100)

In [12]:
X_train.shape

(2269, 128, 216)

# Transformations

### X_train

In [13]:
inputs = []
for n in range(X_train.shape[0]):
    for i in range(211):
        a = []
        a.append([X_train[n,:,i],X_train[n,:,i+1],X_train[n,:,i+2],X_train[n,:,i+3],X_train[n,:,i+4]])
        a = np.asarray(a)
        a = a.reshape(640)
        inputs.append(a)

In [14]:
inputs = np.asarray(inputs)

In [15]:
inputs.shape

(478759, 640)

### X_test

In [16]:
inputs_test = []
for n in range(X_test.shape[0]):
    for i in range(211):
        a = []
        a.append([X_test[n,:,i],X_test[n,:,i+1],X_test[n,:,i+2],X_test[n,:,i+3],X_test[n,:,i+4]])
        a = np.asarray(a)
        a = a.reshape(640)
        inputs_test.append(a)

In [17]:
inputs_test = np.asarray(inputs_test)

In [18]:
inputs_test.shape

(84611, 640)

# Baseline

## Construction as in the paper

In [19]:
def get_model(inputDim):
    """
    define the keras model
    the model based on the simple dense auto encoder 
    (128*128*128*128*8*128*128*128*128)
    """
    inputLayer = Input(shape=(inputDim,))

    h = Dense(128)(inputLayer)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)
    
    h = Dense(8)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(128)(h)
    h = BatchNormalization()(h)
    h = Activation('relu')(h)

    h = Dense(inputDim)(h)

    return Model(inputs=inputLayer, outputs=h)

In [20]:
baseline = get_model(640)

In [21]:
print(baseline.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 640)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               82048     
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)              

In [22]:
baseline.compile(optimizer=Adam(learning_rate = 0.001) , loss='mean_squared_error', metrics=['mae'])

# Training

In [23]:
nb_epochs = 100
bs = 512

### Without tensorboard

In [24]:
baseline.fit(inputs, inputs,
                epochs= nb_epochs,
                batch_size= bs, shuffle =True)

Epoch 1/100
936/936 [==============================] - 20s 22ms/step - loss: 377.1604 - mae: 11.9427
Epoch 2/100
936/936 [==============================] - 20s 22ms/step - loss: 14.9029 - mae: 2.8322
Epoch 3/100
936/936 [==============================] - 17s 19ms/step - loss: 12.1072 - mae: 2.5204
Epoch 4/100
936/936 [==============================] - 18s 19ms/step - loss: 10.8204 - mae: 2.3886
Epoch 5/100
936/936 [==============================] - 17s 18ms/step - loss: 9.7281 - mae: 2.2805
Epoch 6/100
936/936 [==============================] - 17s 18ms/step - loss: 8.7959 - mae: 2.1818
Epoch 7/100
936/936 [==============================] - 17s 18ms/step - loss: 8.3334 - mae: 2.1285
Epoch 8/100
936/936 [==============================] - 18s 19ms/step - loss: 8.1416 - mae: 2.1066
Epoch 9/100
936/936 [==============================] - 19s 20ms/step - loss: 7.8392 - mae: 2.0680
Epoch 10/100
936/936 [==============================] - 17s 18ms/step - loss: 7.6676 - mae: 2.0481
Epoch 11/100


936/936 [==============================] - 17s 18ms/step - loss: 6.3825 - mae: 1.8687
Epoch 84/100
936/936 [==============================] - 17s 18ms/step - loss: 6.3814 - mae: 1.8689
Epoch 85/100
936/936 [==============================] - 19s 20ms/step - loss: 6.3810 - mae: 1.8685
Epoch 86/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3760 - mae: 1.8683
Epoch 87/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3692 - mae: 1.8660
Epoch 88/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3686 - mae: 1.8666
Epoch 89/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3663 - mae: 1.8659
Epoch 90/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3650 - mae: 1.8663
Epoch 91/100
936/936 [==============================] - 18s 19ms/step - loss: 6.3610 - mae: 1.8655
Epoch 92/100
936/936 [==============================] - 19s 20ms/step - loss: 6.3607 - mae: 1.8648
Epoch 93/100
936/936 [=

In [25]:
score = baseline.evaluate(inputs_test, inputs_test)
print(score)

1062/1062 [==============================] - 2s 2ms/step - loss: 6.8236 - mae: 1.9152
[6.823550701141357, 1.9151898622512817]


In [25]:
baseline.save('baseline_slider_allids.h5')